In [1]:

%matplotlib inline

import sklearn
from sklearn.feature_selection import SelectKBest




import gc
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict

plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

In [2]:
station = pd.read_csv('station.csv', low_memory=False)
trip_test = pd.read_csv('trip_test.csv', low_memory=False)
trip_train = pd.read_csv('trip_train.csv', low_memory=False)
weather = pd.read_csv('weather.csv', low_memory=False)


In [3]:
del weather['zip_code']


In [4]:
trip_train.drop('duration', axis=1)

,id,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,907649,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602
1,384043,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,94133
2,316176,6/9/2014 8:42,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107
3,618874,1/26/2015 16:55,San Francisco Caltrain 2 (330 Townsend),69,1/26/2015 17:07,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,94602
4,910977,8/29/2015 15:09,Market at 10th,67,8/29/2015 15:14,Powell Street BART,39,607,Subscriber,94709
5,522083,10/30/2014 7:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,94107
6,880809,8/7/2015 17:11,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,94107
7,488938,10/8/2014 14:10,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,94107
8,899522,8/21/2015 7:53,2nd at Folsom,62,8/21/2015 8:02,San Francisco Caltrain (Townsend at 4th),70,603,Subscriber,94549
9,737380,4/23/2015 6:36,Market at 4th,76,4/23/2015 6:40,Mechanics Plaza (Market at Battery),75,86,Customer,21202


In [5]:
trip_train = pd.merge(trip_train, station[['id', 'lat', 'long']], left_on='start_station_id', right_on= 'id' ,how='left')


In [6]:
trip_train.drop('id_y', axis=1, inplace=True)
trip_train.rename(columns={'id_x' : 'id', 'lat': 'start_station_lat','long': 'start_station_long'}, inplace=True)

In [7]:
trip_train = pd.merge(trip_train, station[['id', 'lat', 'long']], left_on='end_station_id', right_on= 'id' ,how='left')
trip_train

,id_x,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_station_lat,start_station_long,id_y,lat,long
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,37.795392,-122.394203,60,37.804770,-122.403234
1,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,94133,37.776619,-122.417385,46,37.795425,-122.404767
2,316176,334,6/9/2014 8:42,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107,37.789625,-122.400811,62,37.785299,-122.396236
3,618874,666,1/26/2015 16:55,San Francisco Caltrain 2 (330 Townsend),69,1/26/2015 17:07,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,94602,37.776600,-122.395470,55,37.789756,-122.394643
4,910977,318,8/29/2015 15:09,Market at 10th,67,8/29/2015 15:14,Powell Street BART,39,607,Subscriber,94709,37.776619,-122.417385,39,37.783871,-122.408433
5,522083,337,10/30/2014 7:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,94107,37.771058,-122.402717,70,37.776617,-122.395260
6,880809,394,8/7/2015 17:11,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,94107,37.782259,-122.392738,61,37.780526,-122.390288
7,488938,766,10/8/2014 14:10,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,94107,37.788446,-122.408499,70,37.776617,-122.395260
8,899522,531,8/21/2015 7:53,2nd at Folsom,62,8/21/2015 8:02,San Francisco Caltrain (Townsend at 4th),70,603,Subscriber,94549,37.785299,-122.396236,70,37.776617,-122.395260
9,737380,267,4/23/2015 6:36,Market at 4th,76,4/23/2015 6:40,Mechanics Plaza (Market at Battery),75,86,Customer,21202,37.786305,-122.404966,75,37.791300,-122.399051


In [8]:
trip_train.drop('id_y', axis=1, inplace=True)
trip_train.rename(columns={'id_x' : 'id', 'lat': 'end_station_lat','long': 'end_station_long'}, inplace=True)
trip_train

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_station_lat,start_station_long,end_station_lat,end_station_long
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,37.795392,-122.394203,37.804770,-122.403234
1,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,94133,37.776619,-122.417385,37.795425,-122.404767
2,316176,334,6/9/2014 8:42,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107,37.789625,-122.400811,37.785299,-122.396236
3,618874,666,1/26/2015 16:55,San Francisco Caltrain 2 (330 Townsend),69,1/26/2015 17:07,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,94602,37.776600,-122.395470,37.789756,-122.394643
4,910977,318,8/29/2015 15:09,Market at 10th,67,8/29/2015 15:14,Powell Street BART,39,607,Subscriber,94709,37.776619,-122.417385,37.783871,-122.408433
5,522083,337,10/30/2014 7:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,94107,37.771058,-122.402717,37.776617,-122.395260
6,880809,394,8/7/2015 17:11,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,94107,37.782259,-122.392738,37.780526,-122.390288
7,488938,766,10/8/2014 14:10,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,94107,37.788446,-122.408499,37.776617,-122.395260
8,899522,531,8/21/2015 7:53,2nd at Folsom,62,8/21/2015 8:02,San Francisco Caltrain (Townsend at 4th),70,603,Subscriber,94549,37.785299,-122.396236,37.776617,-122.395260
9,737380,267,4/23/2015 6:36,Market at 4th,76,4/23/2015 6:40,Mechanics Plaza (Market at Battery),75,86,Customer,21202,37.786305,-122.404966,37.791300,-122.399051


In [9]:
trip_train['start_station_lat'] = trip_train[pd.to_numeric(trip_train.start_station_lat, errors='coerce').notnull()]['start_station_lat']
trip_train['start_station_long'] = trip_train[pd.to_numeric(trip_train.start_station_long, errors='coerce').notnull()]['start_station_long']
trip_train['end_station_lat'] = trip_train[pd.to_numeric(trip_train.end_station_lat, errors='coerce').notnull()]['end_station_lat']
trip_train['end_station_long'] = trip_train[pd.to_numeric(trip_train.end_station_long, errors='coerce').notnull()]['end_station_long']


In [10]:
from math import radians, cos, sin, asin, sqrt

#distancia entre dos estaciones 
def distanciaEntreEstaciones(long1, lat1, long2, lat2):
    long1, lat1, long2, lat2 = map(radians, [long1, lat1, long2, lat2])
    dLong = long2 - long1 
    dLat = lat2 - lat1 
    a = sin(dLat/2)**2 + cos(lat1) * cos(lat2) * sin(dLong/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

distancia = trip_train.apply(lambda x: distanciaEntreEstaciones(x[11], x[12], x[13], x[14]), axis = 1)
trip_train['distancia_entre_estaciones'] = distancia


In [11]:
trip_train

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_station_lat,start_station_long,end_station_lat,end_station_long,distancia_entre_estaciones
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,37.795392,-122.394203,37.804770,-122.403234,1.148453
1,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,94133,37.776619,-122.417385,37.795425,-122.404767,1.794651
2,316176,334,6/9/2014 8:42,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107,37.789625,-122.400811,37.785299,-122.396236,0.569924
3,618874,666,1/26/2015 16:55,San Francisco Caltrain 2 (330 Townsend),69,1/26/2015 17:07,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,94602,37.776600,-122.395470,37.789756,-122.394643,0.788625
4,910977,318,8/29/2015 15:09,Market at 10th,67,8/29/2015 15:14,Powell Street BART,39,607,Subscriber,94709,37.776619,-122.417385,37.783871,-122.408433,1.084530
5,522083,337,10/30/2014 7:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,94107,37.771058,-122.402717,37.776617,-122.395260,0.892320
6,880809,394,8/7/2015 17:11,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,94107,37.782259,-122.392738,37.780526,-122.390288,0.291147
7,488938,766,10/8/2014 14:10,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,94107,37.788446,-122.408499,37.776617,-122.395260,1.631116
8,899522,531,8/21/2015 7:53,2nd at Folsom,62,8/21/2015 8:02,San Francisco Caltrain (Townsend at 4th),70,603,Subscriber,94549,37.785299,-122.396236,37.776617,-122.395260,0.528155
9,737380,267,4/23/2015 6:36,Market at 4th,76,4/23/2015 6:40,Mechanics Plaza (Market at Battery),75,86,Customer,21202,37.786305,-122.404966,37.791300,-122.399051,0.721470


In [12]:
del trip_train['start_station_lat']
del trip_train['start_station_long']
del trip_train['end_station_lat']
del trip_train['end_station_long']

In [13]:
fechasStart = trip_train['start_date'].map(lambda x: (x.split(' ')[0]))
trip_train['startDate'] =  fechasStart
fechasEnd = trip_train['end_date'].map(lambda x: (x.split(' ')[0]))
trip_train['endDate'] =  fechasEnd

In [14]:
#separamos la fecha por / y eliminamos todo lo q hay mas a la derecha del espacio despues del año
fechas = trip_train['start_date'].map(lambda x: (x.split('/')[0], x.split('/')[1], x.split('/')[2].split(' ')[0]))

#convertimos cada tupla de año mes dia a dia de semana ylo guardamos en un nuevo campo 
trip_train['startDate_weekday'] = fechas.map(lambda x: dt.date(int(x[2]),int(x[0]),int(x[1])).strftime("%A"))

#separamos la fecha por / y eliminamos todo lo q hay mas a la derecha del espacio despues del año
fechas = trip_train['end_date'].map(lambda x: (x.split('/')[0], x.split('/')[1], x.split('/')[2].split(' ')[0]))

#convertimos cada tupla de año mes dia a dia de semana ylo guardamos en un nuevo campo 
trip_train['endDate_weekday'] = fechas.map(lambda x: dt.date(int(x[2]),int(x[0]),int(x[1])).strftime("%A"))

In [15]:
trip_train[trip_train['startDate'] != trip_train['endDate']]

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,distancia_entre_estaciones,startDate,endDate,startDate_weekday,endDate_weekday
62,246208,64803,4/12/2014 17:23,Grant Avenue at Columbus Avenue,73,4/13/2014 11:23,2nd at Townsend,61,364,Customer,7302,2.167697,4/12/2014,4/13/2014,Saturday,Sunday
410,402371,59644,8/10/2014 23:10,Yerba Buena Center of the Arts (3rd @ Howard),68,8/11/2014 15:44,Golden Gate at Polk,59,222,Customer,94102,1.965957,8/10/2014,8/11/2014,Sunday,Monday
984,566595,15091,12/4/2014 22:46,San Francisco City Hall,58,12/5/2014 2:57,Market at 10th,67,387,Customer,92705,0.153497,12/4/2014,12/5/2014,Thursday,Friday
1245,187216,30023,2/17/2014 23:27,Harry Bridges Plaza (Ferry Building),50,2/18/2014 7:47,Embarcadero at Bryant,54,584,Subscriber,94107,0.844848,2/17/2014,2/18/2014,Monday,Tuesday
1331,658718,241766,2/26/2015 2:42,Santa Clara at Almaden,4,2/28/2015 21:52,San Jose Diridon Caltrain Station,2,46,Customer,95126,0.804350,2/26/2015,2/28/2015,Thursday,Saturday
1397,32352,534,9/23/2013 23:57,Embarcadero at Sansome,60,9/24/2013 0:06,Mechanics Plaza (Market at Battery),75,290,Subscriber,94102,0.927041,9/23/2013,9/24/2013,Monday,Tuesday
1654,760403,68367,5/9/2015 23:08,Davis at Jackson,42,5/10/2015 18:08,Mechanics Plaza (Market at Battery),75,523,Customer,94015,0.362559,5/9/2015,5/10/2015,Saturday,Sunday
1664,825527,46016,6/28/2015 13:30,South Van Ness at Market,66,6/29/2015 2:17,Townsend at 7th,65,502,Customer,81475,1.818154,6/28/2015,6/29/2015,Sunday,Monday
1675,635323,71356,2/8/2015 13:47,Embarcadero at Vallejo,48,2/9/2015 9:36,2nd at Townsend,61,442,Customer,979,1.474964,2/8/2015,2/9/2015,Sunday,Monday
1935,535575,458,11/7/2014 23:53,Mountain View Caltrain Station,28,11/8/2014 0:01,Castro Street and El Camino Real,32,102,Subscriber,94040,0.919212,11/7/2014,11/8/2014,Friday,Saturday


In [16]:
#separamos en dos campos nuevos. fecha en formato yyyymmdd y hora del dia en minutos
fechasStart = trip_train['start_date'].map(lambda x: (int(dt.datetime.strptime(x.split(' ')[0], "%m/%d/%Y").strftime("%Y%m%d"))))
horasStart = trip_train['start_date'].map(lambda x: (x.split(' ')[1]))
horasStart = horasStart.map(lambda x: (int(x.split(':')[0]) * 60 *60+ int(x.split(':')[1])*60))

fechasEnd = trip_train['end_date'].map(lambda x: (int(dt.datetime.strptime(x.split(' ')[0], "%m/%d/%Y").strftime("%Y%m%d"))))
horasEnd = trip_train['end_date'].map(lambda x: (x.split(' ')[1]))
horasEnd = horasEnd.map(lambda x: (int(x.split(':')[0]) * 60 *60+ int(x.split(':')[1])*60))

In [17]:
#ver dia y noche como afecta

#separamos la fecha por / y eliminamos todo lo q hay mas a la derecha del espacio despues del año
fechas = trip_train['start_date'].map(lambda x: (x.split('/')[0], x.split('/')[1], x.split('/')[2].split(' ')[0]))

horas = trip_train['start_date'].map(lambda x:x.split('/')[2].split(' ')[1].split(':')[0])
#convertimos cada tupla de año mes dia a dia de semana ylo guardamos en un nuevo campo 

fechas = fechas.map(lambda x: dt.date(int(x[2]),int(x[0]),int(x[1])).strftime("%m"))

i = 0
deNoche = [0] * fechas.size
for line in fechas:
    if(((int(line) < 4) & (int(line) > 0)) | ((int(line) > 9)&(int(line) < 13))):
        if((int(horas[i]) <18) & (int(horas[i]) > 7)):
            deNoche[i] = 0
        else:
            deNoche[i] = 1
    else:
        if((int(horas[i]) <20) & (int(horas[i]) > 6)):
            deNoche[i] = 0
        else:
            deNoche[i] = 1
    i = i + 1 
        
trip_train['startDate_deNoche'] = deNoche

#ver dia y noche como afecta

#separamos la fecha por / y eliminamos todo lo q hay mas a la derecha del espacio despues del año
fechas = trip_train['end_date'].map(lambda x: (x.split('/')[0], x.split('/')[1], x.split('/')[2].split(' ')[0]))

horas = trip_train['end_date'].map(lambda x:x.split('/')[2].split(' ')[1].split(':')[0])
#convertimos cada tupla de año mes dia a dia de semana ylo guardamos en un nuevo campo 

fechas = fechas.map(lambda x: dt.date(int(x[2]),int(x[0]),int(x[1])).strftime("%m"))

i = 0
deNoche = [0] * fechas.size
for line in fechas:
    if(((int(line) < 4) & (int(line) > 0)) | ((int(line) > 9)&(int(line) < 13))):
        if((int(horas[i]) <18) & (int(horas[i]) > 7)):
            deNoche[i] = 0
        else:
            deNoche[i] = 1
    else:
        if((int(horas[i]) <20) & (int(horas[i]) > 6)):
            deNoche[i] = 0
        else:
            deNoche[i] = 1
    i = i + 1 
        
trip_train['endDate_deNoche'] = deNoche

In [18]:
trip_train = pd.merge(trip_train, weather, left_on='startDate', right_on= 'date' ,how='left')
trip_train

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,...,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,...,10.0,10.0,10.0,17.0,6.0,29.0,0,2.0,NaN,272.0
1,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,...,10.0,10.0,10.0,14.0,3.0,20.0,0,1.0,NaN,309.0
2,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,...,20.0,13.0,10.0,17.0,6.0,NaN,0,0.0,NaN,349.0
3,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,...,10.0,10.0,10.0,22.0,5.0,25.0,0,0.0,NaN,356.0
4,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,...,10.0,10.0,10.0,23.0,6.0,29.0,0,3.0,NaN,313.0
5,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,...,10.0,10.0,10.0,21.0,10.0,24.0,0,6.0,NaN,316.0
6,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,...,10.0,10.0,10.0,14.0,5.0,NaN,0,4.0,NaN,345.0
7,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,...,20.0,13.0,7.0,15.0,8.0,NaN,0,5.0,NaN,352.0
8,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,...,10.0,10.0,10.0,16.0,5.0,21.0,0,1.0,NaN,339.0
9,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,...,10.0,10.0,10.0,15.0,5.0,18.0,0,4.0,NaN,294.0


In [19]:
del trip_train['endDate']
del trip_train['startDate']
del trip_train['id']
del trip_train['start_station_name']
del trip_train['end_station_name']
del trip_train['start_date']
del trip_train['end_date']
del trip_train['date']
del trip_train['zip_code']


In [20]:
trip_train['start_date'] = fechasStart
trip_train['end_date'] = fechasEnd
trip_train['start_hour'] = horasStart
trip_train['end_hour'] = horasEnd

In [21]:
#convertimos subscription_type a 1 y 0 
trip_train['duration'] = trip_train[pd.to_numeric(trip_train.duration, errors='coerce').notnull()]['duration']
subscription_type = trip_train['subscription_type'].map(lambda x: 1 if x == 'Subscriber' else 0)
trip_train['subscription_type'] = subscription_type
trip_train.head()

,duration,start_station_id,end_station_id,bike_id,subscription_type,distancia_entre_estaciones,startDate_weekday,endDate_weekday,startDate_deNoche,endDate_deNoche,...,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,start_date,end_date,start_hour,end_hour
0,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,6.0,29.0,0,2.0,NaN,272.0,20150827.0,20150827.0,30960.0,31380.0
1,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,3.0,20.0,0,1.0,NaN,309.0,20140728.0,20140728.0,79560.0,80220.0
2,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,6.0,NaN,0,0.0,NaN,349.0,20140609.0,20140609.0,31320.0,31620.0
3,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,5.0,25.0,0,0.0,NaN,356.0,20150126.0,20150126.0,60900.0,61620.0
4,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,6.0,29.0,0,3.0,NaN,313.0,20150829.0,20150829.0,54540.0,54840.0


In [22]:
trip_train['events'].value_counts()

Rain                 230599
Fog                   78439
Fog-Rain               9664
Rain-Thunderstorm      1451
rain                    916
Name: events, dtype: int64

In [23]:
trip_train['precipitation_inches'] = trip_train['precipitation_inches'].map(lambda x: 0.01 if x == 'T' else x)

In [24]:
#convertimos las clases de la columna events
events = trip_train['events'].map(lambda x: 0 if ((x == 'rain') or (x == 'Rain')) else 1 if x == 'Fog' else 2 if x == 'Fog-Rain' else 3 if x == 'Rain-Thunderstorm' else 4)
trip_train['events'] = events
trip_train.head()

,duration,start_station_id,end_station_id,bike_id,subscription_type,distancia_entre_estaciones,startDate_weekday,endDate_weekday,startDate_deNoche,endDate_deNoche,...,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,start_date,end_date,start_hour,end_hour
0,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,6.0,29.0,0,2.0,4,272.0,20150827.0,20150827.0,30960.0,31380.0
1,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,3.0,20.0,0,1.0,4,309.0,20140728.0,20140728.0,79560.0,80220.0
2,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,6.0,NaN,0,0.0,4,349.0,20140609.0,20140609.0,31320.0,31620.0
3,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,5.0,25.0,0,0.0,4,356.0,20150126.0,20150126.0,60900.0,61620.0
4,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,6.0,29.0,0,3.0,4,313.0,20150829.0,20150829.0,54540.0,54840.0


In [25]:
#convertimos las clases de los dias de semana
startDays = trip_train['startDate_weekday'].map(lambda x: 0 if x == 'Monday' else 1 if x == 'Thursday' 
                                             else 2 if x == 'Wednesday' else 3 if x == 'Tuesday' else 4 if x == 'Friday'
                                            else 5 if x == 'Saturday' else 6)
trip_train['startDate_weekday'] = startDays

endDays = trip_train['endDate_weekday'].map(lambda x: 0 if x == 'Monday' else 1 if x == 'Thursday' 
                                             else 2 if x == 'Wednesday' else 3 if x == 'Tuesday' else 4 if x == 'Friday'
                                            else 5 if x == 'Saturday' else 6)
trip_train['endDate_weekday'] = endDays
trip_train['startDate_weekday'].value_counts()

3    502050
2    492035
1    488755
0    475590
4    449575
5    183865
6    157935
Name: startDate_weekday, dtype: int64

In [26]:
trip_train['endDate_weekday'].value_counts()


3    502125
2    492120
1    488780
0    475830
4    448775
5    183705
6    158470
Name: endDate_weekday, dtype: int64

In [27]:
trip_train['events'].value_counts()

4    2428736
0     231515
1      78439
2       9664
3       1451
Name: events, dtype: int64

In [28]:
trip_train.columns

Index(['duration', 'start_station_id', 'end_station_id', 'bike_id',
       'subscription_type', 'distancia_entre_estaciones', 'startDate_weekday',
       'endDate_weekday', 'startDate_deNoche', 'endDate_deNoche',
       'max_temperature_f', 'mean_temperature_f', 'min_temperature_f',
       'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f',
       'max_humidity', 'mean_humidity', 'min_humidity',
       'max_sea_level_pressure_inches', 'mean_sea_level_pressure_inches',
       'min_sea_level_pressure_inches', 'max_visibility_miles',
       'mean_visibility_miles', 'min_visibility_miles', 'max_wind_Speed_mph',
       'mean_wind_speed_mph', 'max_gust_speed_mph', 'precipitation_inches',
       'cloud_cover', 'events', 'wind_dir_degrees', 'start_date', 'end_date',
       'start_hour', 'end_hour'],
      dtype='object')

In [30]:
del trip_train['bike_id']
del trip_train['mean_temperature_f']
del trip_train['min_temperature_f']
del trip_train['max_dew_point_f']
del trip_train['min_dew_point_f']
del trip_train['mean_dew_point_f']
del trip_train['mean_humidity']
del trip_train['min_humidity']
del trip_train['max_sea_level_pressure_inches']
del trip_train['min_sea_level_pressure_inches']
del trip_train['mean_sea_level_pressure_inches']
del trip_train['wind_dir_degrees']
del trip_train['mean_visibility_miles']
del trip_train['min_visibility_miles']
del trip_train['max_wind_Speed_mph']
del trip_train['max_humidity']

KeyError: 'bike_id'

In [31]:
q = trip_train
#w = test
q


,duration,start_station_id,end_station_id,subscription_type,distancia_entre_estaciones,startDate_weekday,endDate_weekday,startDate_deNoche,endDate_deNoche,max_temperature_f,max_visibility_miles,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,start_date,end_date,start_hour,end_hour
0,396,50,60,1,1.148453,1,1,0,0,84.0,10.0,6.0,29.0,0,2.0,4,20150827.0,20150827.0,30960.0,31380.0
1,396,50,60,1,1.148453,1,1,0,0,91.0,10.0,3.0,20.0,0,1.0,4,20140728.0,20140728.0,79560.0,80220.0
2,396,50,60,1,1.148453,1,1,0,0,87.0,20.0,6.0,NaN,0,0.0,4,20140609.0,20140609.0,31320.0,31620.0
3,396,50,60,1,1.148453,1,1,0,0,89.0,10.0,5.0,25.0,0,0.0,4,20150126.0,20150126.0,60900.0,61620.0
4,396,50,60,1,1.148453,1,1,0,0,92.0,10.0,6.0,29.0,0,3.0,4,20150829.0,20150829.0,54540.0,54840.0
5,636,67,46,1,1.794651,0,0,1,1,72.0,10.0,10.0,24.0,0,6.0,4,20141030.0,20141030.0,25200.0,25560.0
6,636,67,46,1,1.794651,0,0,1,1,78.0,10.0,5.0,NaN,0,4.0,4,20150807.0,20150807.0,61860.0,62220.0
7,636,67,46,1,1.794651,0,0,1,1,78.0,20.0,8.0,NaN,0,5.0,4,20141008.0,20141008.0,51000.0,51780.0
8,636,67,46,1,1.794651,0,0,1,1,79.0,10.0,5.0,21.0,0,1.0,4,20150821.0,20150821.0,28380.0,28920.0
9,636,67,46,1,1.794651,0,0,1,1,82.0,10.0,5.0,18.0,0,4.0,4,20150423.0,20150423.0,23760.0,24000.0


In [32]:
a = q['duration'].mean()
b = q['start_station_id'].mean()
c = q['end_station_id'].mean()
d = q['subscription_type'].mean()
e = q['distancia_entre_estaciones'].mean()
f = q['startDate_weekday'].mean()
g = q['endDate_weekday'].mean()
h = q['startDate_deNoche'].mean()
i = q['endDate_deNoche'].mean()
j = q['max_temperature_f'].mean()
k = q['max_visibility_miles'].mean()
l = q['mean_wind_speed_mph'].mean()
m = q['max_gust_speed_mph'].mean()
#n = q['precipitation_inches'].mean()
o = q['cloud_cover'].mean()
p = q['events'].mean()
r = q['start_date'].mean()
s = q['end_date'].mean()
aa = q['start_hour'].mean()
ab = q['end_hour'].mean()


In [33]:
q1 = q.dropna(how='all')
q1 = q1.replace([np.inf, -np.inf], np.nan)
#q1 = q.fillna(0)
q1['duration'] = q['duration'].fillna(a)
q1['start_station_id'] = q['start_station_id'].fillna(b)
q1['end_station_id'] = q['end_station_id'].fillna(c)
q1['subscription_type'] = q['subscription_type'].fillna(d)
q1['distancia_entre_estaciones'] = q['distancia_entre_estaciones'].fillna(e)
q1['startDate_weekday'] = q['startDate_weekday'].fillna(f)
q1['endDate_weekday'] = q['endDate_weekday'].fillna(g)
q1['startDate_deNoche'] = q['startDate_deNoche'].fillna(h)
q1['endDate_deNoche'] = q['endDate_deNoche'].fillna(i)
q1['max_temperature_f'] = q['max_temperature_f'].fillna(j)
q1['max_visibility_miles'] = q['max_visibility_miles'].fillna(k)
q1['mean_wind_speed_mph'] = q['mean_wind_speed_mph'].fillna(l)
q1['max_gust_speed_mph'] = q['max_gust_speed_mph'].fillna(m)
q1['precipitation_inches'] = q['precipitation_inches'].fillna(0)
q1['cloud_cover'] = q['cloud_cover'].fillna(o)
q1['events'] = q['events'].fillna(p)
q1['start_date'] = q['start_date'].fillna(r)
q1['end_date'] = q['end_date'].fillna(s)
q1['start_hour'] = q['start_hour'].fillna(aa)
q1['end_hour'] = q['end_hour'].fillna(ab)
#q1 = q1.replace(q1.mean(),np.nan)
#w = q.fillna(0)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

import time

for i in range(4):
    q5 = q1
    q5['is_true'] = np.random.uniform(0,1,len(q1)) <= 0.2
    q5 = q5[q5['is_true']==True]
    del q5['is_true']
    x1 = q5.drop('duration'
             , axis=1)
    y1 = q5['duration']
    t = w.drop('duration', axis = 1)

    print("[INFO] tuning hyperparameters via grid search")
    params = [
       {'C': [1,0.1,0.01,10,0.001], 'gamma': [0.001,0.01, 0.1,1,10], 'kernel': ['poly']},
    ]
    model =  SVR()
    grid = GridSearchCV(model, params, n_jobs = -1 )
    start = time.time()
    print("[INFO] fitting stage")

    acc = grid.fit(x1, y1).best_score_

    print("[INFO] grid search took {:.2f} seconds".format(time.time() - start))
    # evaluate the best grid searched model on the testing data
    #acc = grid.score(t, w['duration'])
    print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
    print("[INFO] grid search best parameters: {}".format(grid.best_params_))


M:\Users\Krion\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[INFO] tuning hyperparameters via grid search
[INFO] fitting stage


In [36]:
t1 = test.fillna(0)

In [37]:
del t1['duration']

In [41]:
grid.predict(t1)

array([ 517.89858443,  517.89858443,  517.89859947, ...,  517.89858443,
        517.89858443,  517.89858443])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVR
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn import preprocessing
import time

num = 3
max_Accuracy = 0
for i in range(num):
    r2_scorer = make_scorer(r2_score)
    q5 = q1
    q5['is_true'] = np.random.uniform(0,1,len(trip_train)) <= 0.8
    train, test = q5[q5['is_true']==True], q5[q5['is_true']==False]
    del q5['is_true']
    del train['is_true']
    del test['is_true']
    train_x = train.drop('duration'
             , axis=1)
    train_y = train['duration']
    
    test_x = test.drop('duration'
             , axis=1)
    test_y = test['duration']
    train_x = preprocessing.normalize(train_x)
    test_x = preprocessing.normalize(test_x)
    train_x = preprocessing.scale(train_x)
    test_x = preprocessing.scale(test_x)
    print("[INFO] tuning hyperparameters via grid search")
    params = [
       {'C': [1,0.1,0.01,10,0.001,100,1000], 'epsilon': [0,0.1,1,10]},
    ]
    model =  LinearSVR()
    
    # params = [
    #      { 'average' : [True] , 'loss' : ['epsilon_insensitive', 'squared_loss'], 'penalty': ['elasticnet', 'l1'] ,'learning_rate' : ['optimal', 'invscaling'] , 'epsilon' : [0,0.1,1,2,4], 'l1_ratio' : [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}  
    # ]
    #model = linear_model.SGDRegressor()
    
    grid = GridSearchCV(model, params, n_jobs = -1)
    start = time.time()
    print("[INFO] fitting stage")

    grid = grid.fit(train_x, train_y)
    acc = grid.best_score_
    prediction = grid.best_estimator_.predict(test_x)
    score = mean_squared_error(test_y, prediction)
    
    #acc = grid.best_score_
    #accs = grid.score(x1, y1)
    print("[INFO] grid search took {:.2f} seconds".format(time.time() - start))
    # evaluate the best grid searched model on the testing data
    print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
    print("[INFO] grid search best parameters: {}".format(grid.best_params_))
    print("[INFO] grid search best parameters predictor score: {}".format((1 - abs(score)) * 100))
    #print (accs)
    

[INFO] tuning hyperparameters via grid search
[INFO] fitting stage


In [167]:
la = pd.DataFrame(prediction)
a = la[0]
lm = grid.best_estimator_
mean_squared_error(np.array(test_y),np.array(prediction))

6.5835356409538243e+45

In [156]:
np.array(test_y)

array([396, 334, 666, ..., 466, 293, 293], dtype=int64)

In [138]:
prediction

array([  8.12063227e+22,   8.11618186e+22,   8.11063700e+22, ...,
         8.11389947e+22,   8.11389945e+22,   8.11389944e+22])

In [49]:
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

r2_score(test_y, predictions)
#mean_squared_error(test_y, predictions)
#mean_absolute_error(test_y, predictions)

-7.2197969254080115e+36